In [15]:
from aocutils.common import ints
from dataclasses import dataclass
from collections import namedtuple
import heapq

Robot = namedtuple('Robot', 'l r u p')
lines=open('day23input.txt').read().splitlines()
maxx = 0
robots =[Robot(*ints(line)) for line in lines]
maxx = max(max(r.r, r.l, r.u) + r.p for r in robots)
size = 2
while size < maxx:
    size *= 2
size

536870912

In [16]:
@dataclass
class RoboBlock():
    left:int
    right:int
    up:int
    leftrange:int
    rightrange:int
    uprange:int
    totalhits:int=0
    
    def split(self):
        midl = self.left + self.leftrange //2 + 1
        midr = self.right + self.rightrange //2 + 1
        midu = self.up + self.uprange // 2 + 1
        for l in (self.left, midl):
            for r in (self.right, midr):
                for u in (self.up, midu):
                    yield RoboBlock(l,r,u,self.leftrange//2, self.rightrange//2, self.uprange//2)
   
    def hits(self, other):
        dis = 0
        dis += max(0, self.left - other.l, other.l - (self.left + self.leftrange))
        dis += max(0, self.right - other.r, other.r - (self.right + self.rightrange))
        dis += max(0, self.up - other.u, other.u - (self.up + self.uprange))
        return dis <= other.p
    
    def count(self, others):
        self.distance = (abs(self.left) + abs(self.right) + abs(self.up))
        self.totalhits = -sum(self.hits(o) for o in others)
    
    def __gt__(self,other):
        return (self.totalhits, self.leftrange, self.distance) > (other.totalhits, other.leftrange, other.distance)

In [17]:
b = RoboBlock(0,0,0,size,size,size)
b.count(robots)
queue = [b]

while queue:
    cur = heapq.heappop(queue)
    if cur.leftrange == 0:
        break
    else:
        for b in cur.split():
            b.count(robots)
            heapq.heappush(queue, b)
cur.distance

121493971

In [ ]:
# docs here
# https://z3prover.github.io/api/html/classz3py_1_1_optimize.html 
from collections import defaultdict
from z3 import *
nanobots = [ints(line) for line in lines]
o = Optimize()

def zabs(x):
    return If(x >= 0,x,-x)

(x, y, z) = (Int(ch) for ch in 'x y z'.split())
zero = Int('distance to 0,0,0')
inrange = Int('number of bots in range')
ranges = []


for i, (x2,y2,z2,r) in enumerate(nanobots):
    ranges.append(Int(f'range {i}'))
    o.add(ranges[-1] == If(zabs(x-x2) + zabs(y-y2) + zabs(z-z2) <= r, 1, 0))
    
o.add(zero == zabs(x) + zabs(y) + zabs(z))
o.add(inrange == z3.Sum(ranges))

h1 = o.maximize(inrange)
h2 = o.minimize(zero)
o.check()
m = o.model()
o.lower(h2), o.upper(h2)

m = o.model()
for const in m:
    if not const.name().startswith('range '):
        print(const, m[const])

(121493971, 121493971)

In [31]:
m = o.model()
for const in m:
    if not const.name().startswith('range '):
        print(const, m[const])

number of bots in range 985
y 21678597
distance to 0,0,0 121493971
z 48819396
x 50995978


In [32]:
o.objectives()

[-number of bots in range, distance to 0,0,0]